# 开发情况记录
- 2018-10-30 （1）基本文件读入内存 OK （2）对标人行数据筛查功能 雏形验证 
- 2018-10-31 （1）考虑基本的程序功能组合并采用合理方式表示 考虑传入变长参数
- 2018-11-01 （2）为程序的灵活性，考虑query问题

In [26]:
#------------引入库包----------------
#import os
import gc #释放内存用
import pandas as pd
from pandasql import sqldf


def pysqldf(query):
   
    #使用该函数后可以不再添加locals()参数
    
    return sqldf(query, globals())

#------------赋值常量----------------
CONST_ROWS = 1000 # 读入csv文件时，每次读入CONST_ROWS行处理，避免内存溢出


#-------------先释放前期内存-----------
if 'reader' in locals().keys():
    del reader
    gc.collect()
    print("删除原表")
else:
    print("建立表")

#-------------建立文件对象并读文件-----------
with open('G:/aml/绵阳/提数原始数据/new4.csv','r',encoding='gb18030') as f_read:
#f_read = open('G:/aml/绵阳/提数原始数据/new4.csv','r',encoding='gb18030')
    reader = pd.read_csv(f_read,#此处无法使用f_read.read(size)函数，原因是在pd.read_csv函数中的参数限制了该地方的使用
                         sep = '|',#分割符是:|
                         
                         #chunksize = CONST_ROWS,
                         #使用本参数后则pd.read_csv返回迭代类型TextFileReader给reader，每CONST_ROWS条迭代一次，数据占用较大内存时使用
                         #那么在接下来的代码中需要将reader迭代才能完整表达原csv文件内容
                         iterator=True, #使用迭代，不能与此chunksize同时使用？
                         
                         #———————————以下赋列名（按照人行标准文件接口定义列名）——————————
                         header = None,#与下面names配套使用
                         names = ['Head_no',
                                'Bank_code1',
                                'Cst_no',
                                'Open_time',
                                'Close_time',
                                'Acc_name',
                                'Cst_sex',
                                'Nation',
                                'Id_type',
                                'Id_no',
                                'Id_deadline',
                                'Occupation',
                                'Income',
                                'Contact1',
                                'Contact2',
                                'Contact3',
                                'Address1',
                                'Address2',
                                'Address3',
                                'Company',
                                'Sys_name']
                        )

    #————————————————以下需要对reader迭代处理——————————————————
    chunks = [] #初始化块
#    for r in reader:
#        chunk = r.get_chunk(CONST_ROWS)#get_chunk()必须输入参数表示读多少行？
#        chunks.append(chunk)  
    
#print("Iteration is stopped.")
#df = pd.concat(chunks, ignore_index=True)

    loop = True
    while loop:
        try:
            chunk = reader.get_chunk(CONST_ROWS)
            #chunk2 = chunk[(chunk['Id_deadline']< 20180930)]#筛查出身份证件有效期在20180930之前的客户记录
            #chunk2 = chunk.query('Id_deadline < 20180930') #验证query函数的使用，1、此处为身份证件到期
            
            #chunk2 = chunk.query('Id_no.length() == 15') # 2、一代身份证,未通过，使用query似乎不行
            chunk2 = chunk[chunk['Id_no'].str.len() == 15]
        
            #考虑引入pandasql库后采用sql语言
            #chunk2 = pysqldf('select * from chunk where length(Id_no)=15;') 
            
            chunks.append(chunk2)
            #del chunk
            del chunk2
        except StopIteration:
            loop = False
            print("OK")
    df = pd.concat(chunks, ignore_index=True)#ignore_index 忽略需要连接的frame本身的index。当原本的index没有特别意义的时候可以使用
df.shape




删除原表
OK


(99921, 21)

# 筛查逻辑
## 一、主体结构抽象
### 1.输入
- 
### 2.输出
### 3.实现
## 二、具体筛查模块

In [6]:
! pip install pandasql

  Running setup.py bdist_wheel for pandasql: started
  Running setup.py bdist_wheel for pandasql: finished with status 'done'
  Stored in directory: C:\Users\qlx\AppData\Local\pip\Cache\wheels\53\6c\18\b87a2e5fa8a82e9c026311de56210b8d1c01846e18a9607fc9
Successfully built pandasql


In [16]:
df.head()

,Head_no,Bank_code1,Cst_no,Open_time,Close_time,Acc_name,Cst_sex,Nation,Id_type,Id_no,...,Occupation,Income,Contact1,Contact2,Contact3,Address1,Address2,Address3,Company,Sys_name
0,10011101026226,2000533,20000126066470,20030829,99991231,张代清,11.0,CHN,11,510703480701051,...,其它,0.0,60552111,0,69544216,富丽华大酒家.13911138189,开元路一段3号游仙区工商分局,剑南路东段209号,游仙区工商分局,企业级客户信息
1,10011101026226,2000956,20000261256491,20110903,99991231,王成,11.0,CHN,11,510725700301861,...,工人,0.0,13881123174,None,None,中坝镇体育路一巷41号9幢3单元7楼42号,None,None,None,企业级客户信息
2,10011101026226,10010689,100100016275012,20010704,99991231,张伟,11.0,CHN,11,512921760130575,...,其它,0.0,15901801380,None,None,上海宝山月浦盛星10队2号路,电力混凝土,５１２９２１７６０１３０５７５,"""""""""""""""""""""",企业级客户信息
3,10011101026226,12020223,120200049403593,20040513,99991231,罗梓宁,11.0,CHN,11,510725791004001,...,公司员工,0.0,08168214069,13184228905,None,青年时报发行中心,None,None,None,企业级客户信息
4,10011101026226,16090200,160900026356338,20050909,99991231,曾建,11.0,CHN,11,510724730718121,...,其它,0.0,2239888,None,None,临园路76号,临园路76号,临园路76号,None,企业级客户信息


In [1]:
#2018-11-04 23：54测试git
print('测试提交git')

测试提交git
